<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/train_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')

# 2. 경로 설정
PROJECT_PATH = '/content/drive/MyDrive/AS_LAB'
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)

# 3. 시스템 경로 추가 및 작업 디렉토리 변경
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

print(f"현재 위치: {os.getcwd()}")

Mounted at /content/drive
현재 위치: /content/drive/MyDrive/AS_LAB


In [ ]:
# 파일 이름을 명시적으로 지정
FILE_NAME = "train.py"
# 전체 경로 사용
SAVE_PATH = f"/content/drive/MyDrive/AS_LAB/{FILE_NAME}"

In [ ]:
content = """
import torch
import torch.nn as nn
import torch.optim as optim
import yaml
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
import json

#1. 데이터 로더 및 get_model함수 임포트
from my_datasets import UnifiedDataLoader
from models import get_model

#config 파일을 읽어 딕셔너리로 변환하는 함수
def load_config(config_path):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

#시드 고정해주는 함수(재현성 확보를 위해 구현)
def set_seed(seed):
    torch.manual_seed(seed) #CPU 연산 시드 고정
    np.random.seed(seed) #NumPy 연산 시드 고정
    random.seed(seed) #Python 기본 랜덤 시드 고정
    if torch.cuda.is_available(): #GPU가 있다면
        torch.cuda.manual_seed(seed) #GPU 연산 시드 고정
        torch.cuda.manual_seed_all(seed) #멀티 GPU라면 전체 연산 시드 고정

#loss/accuracy curve 그리는 함수
def plot_curve(history, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(12, 5))

    #1. loss 그래프
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Train Loss', color='blue')
    plt.plot(epochs, history['val_loss'], label='Val Loss', color='red')
    plt.title('Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    #2. Accuracy 그래프
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], label='Train Acc', color='blue')
    plt.plot(epochs, history['val_acc'], label='Val Acc', color='red')
    plt.title('Accuracy Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    #저장
    save_path = os.path.join(save_dir, 'learning_curve.png')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"학습 그래프가 저장되었습니다: {save_path}")

#Val Loss가 더 이상 줄어들지 않으면 학습을 강제 종료하는 클래스
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, path='checkpoint.pth'):
        self.patience = patience #Val Loss가 줄어들지 않아도 참아주는 횟수
        self.verbose = verbose #로그 출력 여부
        self.counter = 0 #Val Loss가 줄어들지 않은 횟수 카운터
        self.best_score = None #현재 최고 점수
        self.early_stop = False #Early Stopping 신호
        self.val_loss_min = np.inf #최소 Val Loss 기록(초기 값은 무한대: 첫 번째 epoch의 결과값을 무조건 최고기록으로 저장하기 위해)
        self.path = path #모델 저장 경로

    def __call__(self, val_loss, model): #객체 실행 함수(매 Epoch마다 실행)
        score = -val_loss #loss가 낮으면 점수가 높게 설정
        if self.best_score is None: #첫 번째 epoch인 경우
            self.best_score = score #현재 점수를 최고 점수로 저장
            self.save_checkpoint(val_loss, model) #모델 저장
        elif score < self.best_score: #성능 개선 실패할 경우
            self.counter += 1 #카운트 1 증가
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience: #참는 횟수 초과 시
                self.early_stop = True #종료 신호 켜기
        else: #성능 개선 성공한 경우
            self.best_score = score #최고 점수 갱신
            self.save_checkpoint(val_loss, model) #모델 저장
            self.counter = 0 #카운트 초기화

    #모델의 가중치 파일로 저장하는 함수
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). 모델을 저장합니다...')
        torch.save(model.state_dict(), self.path) #state_dict(가중치) 저장
        self.val_loss_min = val_loss #최소 loss 갱신

#메인 함수
def main():
    #1. 시드 및 설정 로드
    cfg = load_config('config.yaml') #config 파일 불러오기
    set_seed(cfg['train']['seed']) #시드 고정(config 파일에 지정해둔 시드)
    aug_config = cfg['data'].get('augmentation', {'use_aug': False})
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    base_dir = os.getcwd() #저장 경로 절대 경로로 반환
    result_dir = os.path.join(base_dir, 'results', cfg['project_name'])
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    save_path = os.path.join(base_dir, cfg['train']['save_path'])
    print(f"device: {device} | model: {cfg['model']['type']}")

    #2. 데이터 로더
    data_loader = UnifiedDataLoader(cfg) #통합 데이터 로더 객체 생성
    train_loader = data_loader.get_loader(cfg['data']['dataset_name'], 'train') #학습용 로더 가져오기
    val_loader = data_loader.get_loader(cfg['data']['dataset_name'], 'val') #검증용 로더 가져오기

    #3. 모델 초기화(설정 파일 값들을 이용해 모델 생성)
    dropout_rate = cfg['model'].get('dropout_rate', 0.0) #Dropout 확률 설정
    model = get_model(
        cfg['model']['type'],
        cfg['model']['input_size'],
        cfg['model']['hidden_size'],
        cfg['model']['num_classes'],
        dropout_rate = dropout_rate
    )
    model = model.to(device) #모델을 선택된 장치 메모리로 이동

    #4. 학습 도구 설정
    #Optimizer 선택 및 weight decay 적용
    lr = cfg['train']['learning_rate']
    weight_decay = cfg['train'].get('weight_decay', 0.0) #과적합 방지용(정답 외우는 방식 사용하면 감점)
    optimizer_name = cfg['train'].get('optimizer', 'adam') #optimizer 설정값이 있으면 설정된 기법, 없으면 Adam 기법

    if optimizer_name.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    #스케줄러 설정
    scheduler = None
    if cfg['train'].get('use_scheduler', False):
        step_size = cfg['train'].get('scheduler_step', 10)
        gamma = cfg['train'].get('scheduler_gamma', 0.1)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
        print(f"Scheduler Activated: StepLR (step={step_size}, gamma={gamma})")

    criterion = nn.CrossEntropyLoss() #손실 함수: 다중 분류용 엔트로피(예측과 정답을 비교해서 점수(벌점) 매기는 함수)
    early_stopping = EarlyStopping(patience=cfg['train']['patience'], verbose=True, path=cfg['train']['save_path']) #Early Stopping 도구 준비

    #기록 저장용 딕셔너리
    history = {
        'train_loss' : [],
        'train_acc' : [],
        'val_loss' : [],
        'val_acc' : []
    }

    #5. 학습 루프 시작
    epochs = cfg['train']['num_epochs'] #전체 반복 세대 수
    print('학습을 시작합니다...')

    train_time = 0.0
    for epoch in range(epochs):
        start_time = time.time() #시간 측정 시작(Epoch 시작 직전)
        #train
        model.train() #모델을 학습 모드로 전환()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            train_total +=labels.size(0)
            train_correct += (predicted == labels).sum().item()

        end_time = time.time() #시간 측정 종료(Train Loop 종료시)
        epoch_duration = end_time - start_time #소요 시간 계산(초 단위)

        #평균 Loss 및 정확도 계산
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_correct / train_total

        #validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        #계산
        epoch_duration = time.time() - start_time
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_correct / train_total
        avg_val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total
        train_time += epoch_duration

        #history 저장
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)

        #스케줄러 업데이트
        current_lr = optimizer.param_groups[0]['lr']
        if scheduler:
            scheduler.step()

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Time: {epoch_duration:.2f}s | "
              f"Train Loss: {avg_train_loss:.4f} Train Acc: {avg_train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} Val Acc: {val_acc:.4f}")

        #Early Stopping 확인
        early_stopping(avg_val_loss, model)
        if early_stopping.early_stop:
            print("Early Stopping이 작동되었습니다.")
            break

    print(f"학습이 종료되었습니다. | 학습 시간: {train_time}")

    #결과 저장
    json_path = os.path.join(result_dir, 'history.json')
    with open(json_path, 'w') as f:
        json.dump(history, f)
    #그래프 그리기
    plot_curve(history, save_dir=result_dir)

if __name__ == "__main__":
    main()
"""
try:
    with open(SAVE_PATH, 'w', encoding='utf-8') as f:
        f.write(content.strip())
    print(f"파일이 안전하게 생성되었습니다: {SAVE_PATH}")
except Exception as e:
    print(f"저장 실패: {e}")

파일이 안전하게 생성되었습니다: /content/drive/MyDrive/AS_LAB/train.py
